In [10]:
# zuerst installiere ich Requests, BeautifulSoup und Pandas

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [12]:
# dann hole ich meine HTML-Site und bringe sie mit BeautifulSoup in Form

r = requests.get('https://www.transfermarkt.ch/1-bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2018')
soup = BeautifulSoup(r.text, 'html.parser')

In [13]:
# ich gebe sie aus

soup

<html>
<head><title>404 Not Found</title></head>
<body bgcolor="white">
<center><h1>404 Not Found</h1></center>
<hr/><center>nginx</center>
</body>
</html>

In [14]:
# es gibt eine Fehlermeldung, weil die Seite transfermarkt.ch solche Anfragen blockiert
# Barnabys Code umgeht die Blockade

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
url = 'https://www.transfermarkt.ch/super-league/startseite/wettbewerb/C1/plus/?saison_id=2017'

# ich wiederhole den Vorgang

r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
soup


<!DOCTYPE html>

<!-- paulirish.com/2008/conditional-stylesheets-vs-css-hacks-answer-neither/ -->
<!--[if IE 7]>
<html class="ie7 oldie" lang="de"> <![endif]-->
<!--[if IE 8]>
<html class="no-js lt-ie9" lang="de"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="de"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="x-ua-compatible"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="#1a3151" name="theme-color">
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/favicon-16x16.png" rel="shortcut icon" sizes="16x16"/>
<link href="/android-chrome-192x192.png" rel="shortcut icon" sizes="192x192"/>
<link href="/apple-touch-icon-152x152.png" rel="apple-touch-icon-precomposed"/>
<link href="http://www.transfermarkt.de/super-league/startseite/wettbewerb/C1/saison_id/2017/plus/" hreflang="x-default" rel="alternate">
<link href="http://www.transfermarkt.de/super-league/startseite/wettbewerb/C1/saiso

In [15]:
# jetzt funktionierts
# nächster Schritt: ich ziehe die Vereine raus, indem ich nach allen Elementen mit folgendem Pfad suche 

vereine = soup.find_all('td', {'class':'hauptlink no-border-links show-for-small show-for-pad'})

In [16]:
# ich baue eine for-Schleife, die mir alle Vereine ausspuckt

for team in vereine:
    print(team.text)

FC Basel
FC Sion
BSC Young Boys
Grasshoppers
FC St. Gallen
Lausanne-Sport
FC Luzern
FC Lugano
FC Zürich
FC Thun


In [17]:
# ich ziehe die Marktwerte der Vereine raus, indem ich nach allen Elementen mit folgendem Pfad suche

marktwerte = soup.find_all('td', {'class':'rechts show-for-small show-for-pad nowrap'})

In [18]:
# ich baue eine for-Schleife, die mir alle Marktwerte ausspuckt

for wert in marktwerte:
    print(wert.text)

243,95 Mio. €
681 Tsd. €
59,93 Mio. €
1,54 Mio. €
36,35 Mio. €
865 Tsd. €
25,45 Mio. €
979 Tsd. €
23,20 Mio. €
611 Tsd. €
20,63 Mio. €
573 Tsd. €
19,10 Mio. €
531 Tsd. €
15,73 Mio. €
477 Tsd. €
15,60 Mio. €
459 Tsd. €
15,05 Mio. €
407 Tsd. €
12,93 Mio. €
349 Tsd. €


In [19]:
# die ersten beiden Werte kann ich nicht brauchen, weil es Gesamtwerte der Liga sind > erst ab 3. Element [2:]
# ich brauche nur die Gesamtmarktwerte > jedes zweite Element überspringen [::2]

for wert in marktwerte[2:][::2]:
    print(wert.text)

59,93 Mio. €
36,35 Mio. €
25,45 Mio. €
23,20 Mio. €
20,63 Mio. €
19,10 Mio. €
15,73 Mio. €
15,60 Mio. €
15,05 Mio. €
12,93 Mio. €


In [20]:
# ich will Kommazahlen (float), keine strings > ich ersetze 'Mio €' durch nichts und das Komma durch einen Punkt

for wert in marktwerte[2:][::2]:
    print(float(wert.text.replace(' Mio. €', '').replace(',', '.')))

59.93
36.35
25.45
23.2
20.63
19.1
15.73
15.6
15.05
12.93


In [21]:
# jetzt bringe ich Vereine und Marktwerte zusammen

teamwertliste = []
for team, wert in zip(vereine, marktwerte[2:][::2]):
    team = team.text
    wert = float(wert.text.replace(' Mio. €', '').replace(',', '.'))
    dictionary = {'Team': team,
                 'Wert': wert}
    
    teamwertliste.append(dictionary)

In [22]:
# ich erstelle mit Pandas ein Dataframe

pd.DataFrame(teamwertliste)

,Team,Wert
0,FC Basel,59.93
1,FC Sion,36.35
2,BSC Young Boys,25.45
3,Grasshoppers,23.20
4,FC St. Gallen,20.63
5,Lausanne-Sport,19.10
6,FC Luzern,15.73
7,FC Lugano,15.60
8,FC Zürich,15.05
9,FC Thun,12.93


In [23]:
# jetzt brauche ich noch die Tabelle mit den Punkten
# ich finde alle Tabellen mit dem class 'responsive-table'

alle_tabellen = soup.find_all('div', {'class':'responsive-table'})

In [24]:
# ich will aber nur die zweite Tabelle im HTML-Code > deshalb: [1]

tabelle2 = alle_tabellen[1]

In [25]:
# in dieser Tabelle alle 'tr' finden (einzelne Reihen)

reihen = tabelle2.find_all('tr')

In [26]:
# ich baue eine for-Schleife, die mir alle Reihen ausspuckt

for reihe in reihen:
    print(reihe.text)


#
Verein
Spiele
+/-
Pkt.


1 


BSC Young Boys 
36
43
84


2 


FC Basel  
36
36
69


3 


FC Luzern 
36
0
54


4 


FC Zürich   
36
6
49


5 


FC St. Gallen 
36
-20
45


6 


FC Sion 
36
-3
42


7 


FC Thun 
36
-15
42


8 


FC Lugano 
36
-17
42


9 


Grasshoppers 
36
-9
39


10 


Lausanne-Sport 
36
-21
35



In [27]:
# die erste Reihe ist die Überschrift, deshalb skippen > [1:]
# ich suche alle Elemente unter dem class 'td'
# ich will nur das letzte Element der Liste (Punkte) > [-1]

for reihe in reihen[1:]:
    alle_tds = reihe.find_all('td')
    punkte = alle_tds[-1]
    print(punkte.text)

84
69
54
49
45
42
42
42
39
35


In [28]:
# ich erstelle ein Dictionnary aus Teams und Punkten

teampunkte = {}

for reihe in reihen[1:]:
    alle_tds = reihe.find_all('td')
    punkte = alle_tds[-1]
    team = alle_tds[2]   # Nimm den Teamnamen aus dem dritten Element der Liste mit [2]
    teampunkte[team.text.strip()] = int(punkte.text)   # Setzte das Päärchen aus Key 'Teamname' und Value 'Punktzahl' in die Liste

teampunkte

{'BSC Young Boys': 84,
 'FC Basel': 69,
 'FC Luzern': 54,
 'FC Zürich': 49,
 'FC St. Gallen': 45,
 'FC Sion': 42,
 'FC Thun': 42,
 'FC Lugano': 42,
 'Grasshoppers': 39,
 'Lausanne-Sport': 35}

In [34]:
# ich hänge das Dictionnary an meine Liste von vorher an

teamwertliste = []
for team, wert in zip(vereine, marktwerte[2:][::2]):
    team = team.text
    wert = float(wert.text.replace(' Mio. €', '').replace(',', '.'))
    row = teampunkte[team]
    dictionary = {'Team': team, 'Wert': wert, 'Punkte': row}
    
    teamwertliste.append(dictionary)
    
teamwertliste

[{'Team': 'FC Basel', 'Wert': 59.93, 'Punkte': 69},
 {'Team': 'FC Sion', 'Wert': 36.35, 'Punkte': 42},
 {'Team': 'BSC Young Boys', 'Wert': 25.45, 'Punkte': 84},
 {'Team': 'Grasshoppers', 'Wert': 23.2, 'Punkte': 39},
 {'Team': 'FC St. Gallen', 'Wert': 20.63, 'Punkte': 45},
 {'Team': 'Lausanne-Sport', 'Wert': 19.1, 'Punkte': 35},
 {'Team': 'FC Luzern', 'Wert': 15.73, 'Punkte': 54},
 {'Team': 'FC Lugano', 'Wert': 15.6, 'Punkte': 42},
 {'Team': 'FC Zürich', 'Wert': 15.05, 'Punkte': 49},
 {'Team': 'FC Thun', 'Wert': 12.93, 'Punkte': 42}]

In [35]:
# ich erstelle mit Pandas eine neue Version des Dataframes mit Punkten und Marktwert

pd.DataFrame(teamwertliste)

,Punkte,Team,Wert
0,69,FC Basel,59.93
1,42,FC Sion,36.35
2,84,BSC Young Boys,25.45
3,39,Grasshoppers,23.20
4,45,FC St. Gallen,20.63
5,35,Lausanne-Sport,19.10
6,54,FC Luzern,15.73
7,42,FC Lugano,15.60
8,49,FC Zürich,15.05
9,42,FC Thun,12.93


In [36]:

## Diese Tabelle will ich nicht nur für die Saison 2017, sondern für mehrere Jahre
# ich setze einen range fest

jahre = list(range(2010,2018))

# und passe die URL an, die ohne Jahresangabe bei allen gleich ist > id=...

url = "https://www.transfermarkt.ch/super-league/startseite/wettbewerb/C1/plus/?saison_id="

In [37]:
# ich baue eine for-Schleife, die mir alle Jahre + URL ausspuckt

for jahr in jahre:
    print(jahr)
    print(url+str(jahr))

2010
https://www.transfermarkt.ch/super-league/startseite/wettbewerb/C1/plus/?saison_id=2010
2011
https://www.transfermarkt.ch/super-league/startseite/wettbewerb/C1/plus/?saison_id=2011
2012
https://www.transfermarkt.ch/super-league/startseite/wettbewerb/C1/plus/?saison_id=2012
2013
https://www.transfermarkt.ch/super-league/startseite/wettbewerb/C1/plus/?saison_id=2013
2014
https://www.transfermarkt.ch/super-league/startseite/wettbewerb/C1/plus/?saison_id=2014
2015
https://www.transfermarkt.ch/super-league/startseite/wettbewerb/C1/plus/?saison_id=2015
2016
https://www.transfermarkt.ch/super-league/startseite/wettbewerb/C1/plus/?saison_id=2016
2017
https://www.transfermarkt.ch/super-league/startseite/wettbewerb/C1/plus/?saison_id=2017


In [38]:
# jetzt führe ich alles zusammen (und nehme beim dictionnary noch das Jahr rein):

allejahre = []
jahre = list(range(2010,2018))

for jahr in jahre:
    url = "https://www.transfermarkt.ch/super-league/startseite/wettbewerb/C1/plus/?saison_id="
    url = (url+str(jahr))
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
           
    for team, wert in zip(vereine, marktwerte[2:][::2]):
        team = team.text
        wert = float(wert.text.replace(' Mio. €', '').replace(',', '.'))
        row = teampunkte[team]
        dictionary = {'Team': team, 'Wert': wert, 'Punkte': row, 'Jahr': jahr}
    
        allejahre.append(dictionary)

In [39]:
# und erstelle mit Pandas eine neue Version des Dataframes mit allen Jahren

pd.DataFrame(allejahre)

,Jahr,Punkte,Team,Wert
0,2010,69,FC Basel,59.93
1,2010,42,FC Sion,36.35
2,2010,84,BSC Young Boys,25.45
3,2010,39,Grasshoppers,23.20
4,2010,45,FC St. Gallen,20.63
5,2010,35,Lausanne-Sport,19.10
6,2010,54,FC Luzern,15.73
7,2010,42,FC Lugano,15.60
8,2010,49,FC Zürich,15.05
9,2010,42,FC Thun,12.93


In [40]:
# ich speichere die Tabelle in eine CSV-Datei

pd.DataFrame(allejahre).to_csv('Projekt_Marktwert.csv', index=False)

In [43]:
# ich lese die CSV-Datei ein und gebe sie aus

df = pd.read_csv('Projekt_Marktwert.csv')
df

,Jahr,Punkte,Team,Wert
0,2010,69,FC Basel,59.93
1,2010,42,FC Sion,36.35
2,2010,84,BSC Young Boys,25.45
3,2010,39,Grasshoppers,23.20
4,2010,45,FC St. Gallen,20.63
5,2010,35,Lausanne-Sport,19.10
6,2010,54,FC Luzern,15.73
7,2010,42,FC Lugano,15.60
8,2010,49,FC Zürich,15.05
9,2010,42,FC Thun,12.93


In [44]:
# ich erstelle eine neue Spalte, die mir anzeigt, wieviel Geld ein Punkt "gekostet" hat
# dazu rechne ich den Marktwert durch Punkte

df['geld_pro_punkt'] =  df['Wert'] / df['Punkte']

In [45]:
df

,Jahr,Punkte,Team,Wert,geld_pro_punkt
0,2010,69,FC Basel,59.93,0.868551
1,2010,42,FC Sion,36.35,0.865476
2,2010,84,BSC Young Boys,25.45,0.302976
3,2010,39,Grasshoppers,23.20,0.594872
4,2010,45,FC St. Gallen,20.63,0.458444
5,2010,35,Lausanne-Sport,19.10,0.545714
6,2010,54,FC Luzern,15.73,0.291296
7,2010,42,FC Lugano,15.60,0.371429
8,2010,49,FC Zürich,15.05,0.307143
9,2010,42,FC Thun,12.93,0.307857


In [46]:
# ich sortiere das Dataframe gemäss meiner neuen Spalte 'geld-pro-punkt'

df.sort_values('geld_pro_punkt', ascending=False).head(10)

,Jahr,Punkte,Team,Wert,geld_pro_punkt
0,2010,69,FC Basel,59.93,0.868551
10,2011,69,FC Basel,59.93,0.868551
70,2017,69,FC Basel,59.93,0.868551
60,2016,69,FC Basel,59.93,0.868551
50,2015,69,FC Basel,59.93,0.868551
30,2013,69,FC Basel,59.93,0.868551
20,2012,69,FC Basel,59.93,0.868551
40,2014,69,FC Basel,59.93,0.868551
51,2015,42,FC Sion,36.35,0.865476
71,2017,42,FC Sion,36.35,0.865476


## Fehler: Es nimmt mir für jede Saison die Punkte und Martkwerte von 2017/2018 !!